# 目前暫時沒用到
### 定義拍子的距離函數

In [11]:
import pickle
class TempoBeats:  
    def save_beats(self,musicname,staff_id,sig,tempo,start,end,beats):
        self.musicname = musicname
        self.staff_id = staff_id
        self.sig = sig
        self.tempo = tempo
        self.start = start
        self.end = end
        self.beats = beats
        
    def save(self,pathname):
        f = open(pathname,'wb')
        savedata = (self.musicname,self.staff_id,self.sig,self.tempo,self.start,self.end,self.beats)
        pickle.dump(savedata,f)
        f.close()
        
    def load(self,pathname):
        f = open(pathname,'rb')
        loaddata = pickle.load(f)
        f.close()
        (self.musicname,self.staff_id,self.sig,self.tempo,self.start,self.end,self.beats) = loaddata
        

In [12]:
m1 = TempoBeats()
m1.save_beats('aaa',1,'4/4',120,1,4,'0.25,0;0.5,1;0.125,1;0.125,1')
pathname = 'E:/python/m1'
m1.save(pathname)

In [14]:
obj = TempoBeats()
obj.load('E:/python/m1')
print obj.musicname

aaa


# ==============================================================

## Basic data and Initialization

In [35]:
import numpy as np
from StringIO import StringIO
measure_1 = {'musicname':'aaa','staff_id':1,'sig':'4/4','tempo':120,'start':1,'end':4,'beat':'0.25,0;0.5,1;0.125,1;0.125,1'}
measure_2 = {'musicname':'aaa','staff_id':1,'sig':'4/4','tempo':120,'start':1,'end':4,'beat':'0.125,1;0.125,1;0.125,1;0.125,1;0.25,1;0.25,1'}

beat1 =  np.array([a[0] for a in np.genfromtxt(StringIO(measure_1['beat'].replace(';','\n')),delimiter = ",")])
beat1_sig = measure_1['sig']
beat2 = np.array([a[0] for a in np.genfromtxt(StringIO(measure_2['beat'].replace(';','\n')),delimiter = ",")])
beat2_sig = measure_2['sig']
print beat1
print beat2

[ 0.25   0.5    0.125  0.125]
[ 0.125  0.125  0.125  0.125  0.25   0.25 ]


## Checking and Defining the distance function
 - 速度差異越大距離越遠
 - 速度本身越快權重越高
 - 依照拍號的重音位置調整  ref: http://jeremydrums.pixnet.net/blog/post/53459686-%E6%8B%8D%E5%AD%90%E8%88%87%E9%9F%B3%E6%A8%82%E9%87%8D%E9%9F%B3

In [53]:
def dis_count(x1,x2,now1,now2,sig):#定義拍子的距離函數!!
    status = now1 - now2
    dic_sig1 = {'4':np.array([2,1,1.5,1]),'2':np.array([2,1]),'3':np.array([2,1,1]),\
                '5':np.array([2,1,2,1,1]),'6':np.array([2,1,1,1.5,1,1]),'7':np.array([2,1,1.5,1,2,1,1])} # 小節的重音權重
    one_beat = int(sig.split('/')[1]) # sig
    here = math.floor(min(now1,now2) * one_beat % one_beat) # position in each measure
    weight = dic_sig1[sig.split('/')[0]][here] # return the weight of the specified position
    if status == 0:
        return 0
    else:
        b_count = 1/min(x1,x2)**2 * abs(x1-x2) * weight
        return b_count

In [54]:
print (np.sum(beat2)==1 and np.sum(beat2)==1) and (beat1_sig == beat2_sig)

p1 = 0
now1 = beat1[p1]
p2 = 0
now2 = beat2[p2]

distance = 0
while(now1 < 1 or now2 < 1):
#     print beat1[p1],beat2[p2]
#     print now1,now2
#     print '--------------------------'
    distance += dis_count(beat1[p1],beat2[p2],now1,now2,beat1_sig)
    if now1 == now2:
        p1 += 1
        p2 += 1
        now1 += beat1[p1]
        now2 += beat2[p1]
    elif now1 > now2:
        p2 += 1
        now2 += beat2[p2]
    else:
        p1 += 1
        now1 += beat1[p1]
print distance

True
76.0


### random 節拍產生器 (測試用)

In [63]:
import random as rd
possible = [1.0,1.0/2,1.0/4,1.0/8,1.0/16]
n = len(possible)
total = 0
beats = []
while (total == 1):
    ran = rd.randint(0,4)
    beats.append(possible[ran])
    total += possible[ran]
    